In [1]:
%matplotlib inline
import PartExtractor.DeformationField as DeformationField
import PartExtractor.DNerfManager as DNerfManager
from run_dnerf import config_parser, create_nerf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
from load_blender import pose_spherical
from run_dnerf import render_path
from run_dnerf_helpers import to8b
from scipy.spatial.transform import Rotation as R
import numpy as np
import cv2 as cv

In [2]:
config_file = "configs/ball_cube_2.txt"
manager = DNerfManager.DNerfManager(config_file = config_file)
times = torch.Tensor([0.0, 0.25, 0.5, 0.75, 1.0])

NeRF type selected: direct_temporal
Found ckpts ['./logs/ball_cube_frame_fix_2/050000.tar', './logs/ball_cube_frame_fix_2/100000.tar', './logs/ball_cube_frame_fix_2/150000.tar', './logs/ball_cube_frame_fix_2/200000.tar', './logs/ball_cube_frame_fix_2/250000.tar', './logs/ball_cube_frame_fix_2/300000.tar', './logs/ball_cube_frame_fix_2/350000.tar', './logs/ball_cube_frame_fix_2/400000.tar', './logs/ball_cube_frame_fix_2/450000.tar', './logs/ball_cube_frame_fix_2/500000.tar', './logs/ball_cube_frame_fix_2/550000.tar', './logs/ball_cube_frame_fix_2/600000.tar', './logs/ball_cube_frame_fix_2/650000.tar', './logs/ball_cube_frame_fix_2/700000.tar', './logs/ball_cube_frame_fix_2/750000.tar', './logs/ball_cube_frame_fix_2/800000.tar', './logs/ball_cube_frame_fix_2/latest.tar']
Reloading from ./logs/ball_cube_frame_fix_2/latest.tar


In [5]:
field = DeformationField.DeformationField(manager, times, torch.Tensor([[-1.0,1.0],[-1.0,1.0],[-1.0,1.0]]), torch.Tensor([10,10,10]).to(dtype=torch.int32))